In [37]:
#supress warnings
import warnings
warnings.filterwarnings('ignore')

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [39]:
#load mnist data
from torchvision import datasets, transforms

def load_data(batch_size=64):
    """
    Loads the MNIST data set and normalizes it to have mean 0 and standard deviation 1.
    """
    # transform data to tensor
    transform = transforms.ToTensor()
    # load data
    train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    # create dataloader
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)
    return train_loader, test_loader

In [40]:
# load data
train_loader, test_loader = load_data()

In [41]:
# define model
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = torch.nn.Dropout2d()
        self.fc1 = torch.nn.Linear(320, 50)
        self.fc2 = torch.nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [42]:
# define loss function and optimizer
model = Model()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.NLLLoss()


In [43]:
# train model
def train(model, loss_fn, optimizer, train_loader, epochs=1):
    """
    Trains the model for one epoch.
    """
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(torch.float), target.to(torch.long)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

In [46]:
# test model
def test(model, loss_fn, test_loader):
    """
    Tests the model on the test data.
    """
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(torch.float), target.to(torch.long)
            output = model(data)
            test_loss += loss_fn(output, target).item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss, correct / len(test_loader.dataset)


In [47]:
# train model
train(model, loss_fn, optimizer, train_loader, epochs=1)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.303723
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.507577
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.535223
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.342416
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.360114
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.369825
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.284101
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.403037
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.443535
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.359689


In [48]:
# test model
test(model, loss_fn, test_loader)

Test set: Average loss: 0.0014, Accuracy: 9728/10000 (97%)


(0.0014210671538952737, 0.9728)